## Using DeepLabV3 To The Steel Data

Source:
https://keras.io/examples/vision/deeplabv3_plus/

In [12]:
import os
import torch
import cv2
import numpy as np
from glob import glob
from scipy.io import loadmat
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

import torch
from torch.utils.data.dataset import Dataset  # For custom data-sets
from torchvision import transforms
from torch.utils.data import DataLoader
import torchvision.transforms.functional as TF
from PIL import Image
import glob
import random

In [13]:
# Miscellaneous
import numpy as np
import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
import os
import shutil
import cv2
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [14]:
def my_image_mask_generator(image_data_generator, mask_data_generator):
    train_generator = zip(image_data_generator, mask_data_generator)
    for (img, mask) in train_generator:
        yield (img, mask)

In [15]:
path = os.getcwd()
batch_size = 2
seed = 42


# Normalize the color values between 0 and 1
image_datagen = ImageDataGenerator(rescale=1/255,
#rotation_range = 20,
#width_shift_range = 0.1,
#height_shift_range = 0.1,
#shear_range = 10,
#horizontal_flip = True,
)

mask_datagen = ImageDataGenerator(width_shift_range=0.1,
                 height_shift_range=0.1) 


validation_generator = ImageDataGenerator(rescale=1/255)

height = 8
width = 8


# Takes the path to a directory & generates batches of augmented data.
train_generator  = (image_datagen.flow_from_directory( path + '/Data/segmentation/train/',
                                          target_size=(height,width),   
                                          color_mode='grayscale',
                                          batch_size = batch_size,
                                          seed = seed,
                                          class_mode = 'categorical'))

train_mask_generator =( mask_datagen.flow_from_directory(path + '/Data/segmentation/train_mask/',
                                          target_size=(height,width),
                                          color_mode='grayscale',
                                          batch_size = batch_size,
                                          seed = seed,
                                          class_mode = 'categorical'))


# Takes the path to a directory & generates batches of augmented data.
validation_generator = (image_datagen.flow_from_directory(path + '/Data/segmentation/test/',
                                          target_size=(height,width),
                                          color_mode='grayscale',
                                          batch_size = batch_size,
                                          class_mode = 'categorical'))

validation_mask_generator =( mask_datagen.flow_from_directory(path + "/Data/segmentation/test_mask/",
                                          target_size=(height,width),
                                          color_mode='grayscale',
                                          batch_size = batch_size,
                                          seed = seed,
                                          class_mode = 'categorical'))




#train = tf.convert_to_tensor(train_generator, dtype=tf.float32)
#test = tf.convert_to_tensor(train_mask_generator, dtype=tf.float32)

#train = zip(train_generator, train_mask_generator)
#test = zip(validation_generator, validation_mask_generator)



train = (pair for pair in zip(train_generator, train_mask_generator))
test = (pair for pair in zip(train_generator, train_mask_generator))

#dataset = tf.data.Dataset.from_generator(train).batch(batch_size)
#train = tf.convert_to_tensor(train, dtype=tf.float32)
#test = tf.convert_to_tensor(train, dtype=tf.float32)

# Create custom generator for training images and masks
#my_train = tuple(my_image_mask_generator(train_generator, train_mask_generator))
#my_test = tuple(my_image_mask_generator(validation_generator, validation_mask_generator))

Found 4968 images belonging to 4 classes.
Found 4969 images belonging to 4 classes.
Found 2129 images belonging to 4 classes.
Found 2129 images belonging to 4 classes.


In [16]:
train

<generator object <genexpr> at 0x2b8452a50>

In [17]:
#dataset = tf.data.Dataset.from_tensor_slices(train)
#dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
#dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [18]:
(x1,y1) ,(x2, y2) =next(train)
len(x1), len(y1), len(y2), len(x2)

(2, 2, 2, 2)

In [19]:
#x1[0].shape

In [20]:
type(train)

generator

In [21]:
# Iterate and see the pictures and labels
img_batch, labels = next(validation_generator)
print('Shape of the input batch' + str(img_batch.shape))
#print('Min of labels: ' + str(min(labels)) + ' and Max of labels: ' +str(max(labels)))

Shape of the input batch(2, 8, 8, 1)


In [22]:
batch_x, batch_y = next(iter(train_generator))
print('y shape    ', batch_y.shape)
print('x shape    ', batch_x.shape)

batch_x, batch_y = next(iter(train_mask_generator))
print('x shape    ', batch_x.shape)

y shape     (2, 4)
x shape     (2, 8, 8, 1)
x shape     (2, 8, 8, 1)


In [23]:
batch_x, batch_y = next(iter(validation_generator))
print('y shape    ', batch_y.shape)
print('x shape    ', batch_x.shape)
print(sorted(np.unique(np.array(batch_y))))

y shape     (2, 4)
x shape     (2, 8, 8, 1)
[0.0, 1.0]


In [24]:

batch_x, batch_y = next(iter(validation_mask_generator))
print('y shape    ', batch_y.shape)
print('x shape    ', batch_x.shape)
print(sorted(np.unique(np.array(batch_y))))

y shape     (2, 4)
x shape     (2, 8, 8, 1)
[0.0, 1.0]


In [25]:
#print("Train Dataset:", train)
#print("Val Dataset:", validation)

In [26]:
def convolution_block(
    block_input,
    num_filters=256,
    kernel_size=3,
    dilation_rate=1,
    padding="same",
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output

In [27]:

def DeeplabV3Plus(image_size, num_classes):
    model_input = keras.Input(shape=(image_size, image_size, 3))
    resnet50 = keras.applications.ResNet50(
        weights="imagenet", include_top=False, input_tensor=model_input
    )
    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same")(x)
    return keras.Model(inputs=model_input, outputs=model_output)

NUM_CLASSES = 4
IMAGE_SIZE = 128


model = DeeplabV3Plus(image_size=IMAGE_SIZE, num_classes=NUM_CLASSES)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1_conv[0][0]']         

In [28]:
# Iterate and see the pictures and labels
#train_img_x, train_mask_y =next(my_train)
#test_img_x, test_mask_y =next(my_test)

In [29]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.01),
    loss=loss,
    metrics=["accuracy"],
)


history = model.fit(train, validation_data= test, epochs=2, verbose=1)

plt.plot(history.history["loss"])
plt.title("Training Loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.show()

plt.plot(history.history["accuracy"])
plt.title("Training Accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.show()

plt.plot(history.history["val_loss"])
plt.title("Validation Loss")
plt.ylabel("val_loss")
plt.xlabel("epoch")
plt.show()

plt.plot(history.history["val_accuracy"])
plt.title("Validation Accuracy")
plt.ylabel("val_accuracy")
plt.xlabel("epoch")
plt.show()

Epoch 1/2


2022-02-16 10:32:01.488184: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


ValueError: in user code:

    File "/Users/michael/neuefische/CapstoneProject_SteelDefectDetection/.venv/lib/python3.9/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/Users/michael/neuefische/CapstoneProject_SteelDefectDetection/.venv/lib/python3.9/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/michael/neuefische/CapstoneProject_SteelDefectDetection/.venv/lib/python3.9/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/Users/michael/neuefische/CapstoneProject_SteelDefectDetection/.venv/lib/python3.9/site-packages/keras/engine/training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "/Users/michael/neuefische/CapstoneProject_SteelDefectDetection/.venv/lib/python3.9/site-packages/keras/utils/tracebac